**Ver o que tem sobre o assunto:**

`1 - Az Minas;`

`2 - Gênero e Número;`

`3 - Twitter;`

`4 - ver como é feita as análises na Novelo.`


**O passo a passo:**

1º - Pegar os tweets;

    https://colab.research.google.com/drive/1QR013vOUjA6Wf4MPYFxh01vsZ91ynUcB?authuser=3#scrollTo=fC2vg0EgTCh8
    
    https://drive.google.com/drive/u/3/my-drive/tweets_aborto.csv
    
    

2º - Fazer a limpeza;

* **Excluir outras linguas;**
* **Excluir duplicados;**
* **Excluir 2022;**
* **Stopwords;**
* ...

**3º - Gerar nuvem de palavras;**

**4º - Separar base de treino;**

**5º - Fazer análise de sentimento;**

**6º - Gráfico das análises;**

7º - Fazer o texto;

8º - Entrevistar alguém;

9º - Pedir meninas para revisarem;

10º - Publicar no site da Carina;

11º - Subir no GitHub.

In [ ]:
# ver quais bibliotecas são do Python e mudar a ordem delas

import altair as alt
import csv
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

from nltk import stem
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
from sklearn.svm import LinearSVC
from PIL import Image
from os import path
from wordcloud import WordCloud, STOPWORDS

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords

### Limpeza dos dados

Aborto é uma palavra comum a muitos idiomas, é necessário fazer a limpeza dos dados e selecionar apenas os tweets em que a língua é "português. Alám disso, ao coletar os dados do Twitter, vieram alguns repetidos e precisamos ficar com dados únicos.

In [ ]:
df = pd.read_csv("tweets_aborto.csv", delimiter = ",")

In [ ]:
df.head()

In [ ]:
# Excluir tweets que não são em português

df_remove = df.loc[(df["language"] != "pt")]

dados = df.drop(df_remove.index)

dados.reset_index(drop = True)

# Excluir duplicados

dados = dados.drop_duplicates(inplace = False).reset_index(drop = True)
dados

### Pré-processamento e tratamento dos dados

Remover as palavras que não possuem relevância e a pontuação, eles não são usados no processamento da linguagem.

Exemplo de stop words: 

In [ ]:
dados["tweet"]

texto = ""
for i, r in dados.iterrows():
    texto = texto + r["tweet"] + "."
    texto = texto.lower()
print(texto)

#### Tokenização

Transformar as palavras em tokens únicos com o objetivo de fazer a contagem...

In [ ]:
tokenizar_p = word_tokenize(texto)

#### Distribuição

Entender como está a distribuição das palavras para limpar.

In [ ]:
fdist = FreqDist(tokenizar_p)
fdist.most_common(10)

In [ ]:
fdist.plot(30, cumulative = False)
plt.show()

#### Stopword

Remover as palavras que não possui significado para o processamento de linguagem natural

In [ ]:
stop_words = set(stopwords.words("portuguese"))
print(stop_words)

In [ ]:
filtrado = []

for x in ["q", "pra", "t", "co", "https", "vc"]:
    stop_words.add(x)
    
for p in tokenizar_p:
    if p not in stop_words:
        filtrado.append(p)

token = RegexpTokenizer(r"\w+")
palavras = token.tokenize(" ".join(filtrado))
print(palavras)

In [ ]:
fdist = FreqDist(palavras)
fdist.most_common(30)

In [ ]:
fdist.plot(30, cumulative = False)
plt.show()

In [ ]:
# Salvar em dataframe

df_palavras = fdist.most_common(40)
df_palavras = pd.DataFrame(df_palavras, columns = ["Palavras", "Quantidade"]).sort_values(by=["Quantidade"],
                                                                                         ascending = False)
# Excluir a palavra aborto
df_palavras = df_palavras.drop(0)

In [ ]:
df_palavras

### Análise de sentimento

...

In [ ]:
# Funções para bag-of-words

def divide(dados):
    dados_new = []
    for palavra in dados:
        palavra_filter = [i.lower() for i in palavra.split()]
        dados_new.append(palavra_filter)
    return dados_new

def bag_of_words(palavras):
    return dict([(palavra, palavras.count(palavra)) for palavra in palavras])

In [ ]:
# Função para treinar os classificadores

def treina_classificadores(positivo, negativo, neutro):
    posdados = []
    with open(positivo, "r", encoding = "utf-8") as myfile:
        ler = csv.reader(myfile, delimiter = ",")
        for val in ler:
            if len(val) > 0:
                posdados.append(val[0])
            
    negdados = []
    with open(negativo, "r", encoding = "utf-8") as myfile:
        ler = csv.reader(myfile, delimiter = ",")
        for val in ler:
            if len(val) > 0:
                negdados.append(val[0])
    
    neudados = []
    with open(neutro, "r", encoding = "utf-8") as myfile:
        ler = csv.reader(myfile, delimiter = ",")
        for val in ler:
            if len(val) > 0:
                neudados.append(val[0])
    
    contra = [(bag_of_words(f), "contra") for f in divide(negdados)]
    pro = [(bag_of_words(f), "a_favor") for f in divide(posdados)]
    outros = [(bag_of_words(f), "outros") for f in divide(neudados)]
    treino = contra + pro + outros
    
    classificadorME = MaxentClassifier.train(treino, 'GIS', trace=0, encoding=None, labels=None,
                                             gaussian_prior_sigma=0, max_iter = 1)
    
    classificadorSVM = SklearnClassifier(LinearSVC(), sparse=False)
    classificadorSVM.train(treino)
    
    classificadorNB = NaiveBayesClassifier.train(treino)
    
    return ([classificadorME, classificadorSVM, classificadorNB])

In [ ]:
# Função para classificar

def classifica(sentencas, classificadores):
    ret = []
    for s in sentencas:
        c = divide([s])
        feats = bag_of_words(c[0])
        classificacao = []
        classificacao.append(classificadores[1].classify(feats))
        classificacao.append(classificadores[2].classify(feats))
        classificacao.append(classificadores[0].classify(feats))
        ret.append(classificacao)
    return ret

In [ ]:
# Base de treino com amostras dos tweets nas polaridades

contra = "tweets_contra.csv"
a_favor = "tweets_a_favor.csv"
outros = "tweets_outros.csv"

classificadores = treina_classificadores(a_favor, contra, outros)


# CLassificação

tweets = dados["tweet"]

classificacao = classifica(tweets, classificadores)


# DataFrame com as classificações

ME = []
SVM = []
NB = []

for cla in classificacao:
    ME.append(cla[0])
    SVM.append(cla[1])
    NB.append(cla[2])
    
df = pd.DataFrame(list(zip(tweets, ME, SVM, NB)), columns = ["tweet", "ME", "SVM", "NB"])

#### Teste para ver qual dos classificadores funcionou melhor

In [ ]:
df.sample(50)

In [ ]:
print(df["tweet"].loc[7879])

A partir da análise, foi possível perceber que o Maxima Entropia (ME) é o que melhor funciona para a nossa amostra, isso porque, de acordo com os testes feitos, ele conseguiu captar melhor que os outros classificadores quando era ironia ou xingamento.

Das 50 amostras analisadas:

* Os três classificadores erraram em 4 afirmações.
* Máxima Entropia (ME) - acertou 39;
* Support Vector Machine (SVM) - acertou 25;
* Naive Bayes (NB) - acertou 36.


**Exemplos**

*Realidade dura: isso não é aborto, é outra coisa lamentável mas não é o absurdo do aborto*

* ME classificou como "contra", SVM e NB classificaram como "a favor".

*Vocês apoiam o aborto?*

* ME classificou como "outros", SVM classificou "contra" e NB classificaram como "a favor".

*Falling é uma oneshot inspirada na música do Harry Styles que conta a história de um casal que acredita ser pra sempre um na vida do outro. No entanto a vida parece não colaborar com o desejo dos dois e tudo parece desmoronar  (Contém menção de aborto)*

* ME classificou como "outros", SVM e NB classificaram como "a favor".

In [ ]:
# Contagem:

df["ME"].value_counts()

In [ ]:
df["SVM"].value_counts()

In [ ]:
df["NB"].value_counts()

In [ ]:
# Dataframes com tweets a favor e outro com tweets contra

pro = df.query("ME == 'a_favor'")
pro = pro.drop(columns = ["SVM", "NB"])

contra = df.query("ME == 'contra'")
contra = contra.drop(columns = ["SVM", "NB"])

In [ ]:
# Palavras mais comuns a favor do aborto:

pro["tweet"]

texto_pro = ""
for i, r in pro.iterrows():
    texto_pro = texto_pro + r["tweet"] + "."
    texto_pro = texto_pro.lower()

token_pro = word_tokenize(texto_pro)

filtrado_pro = []
    
for p in token_pro:
    if p not in stop_words:
        filtrado_pro.append(p)

token_pro = RegexpTokenizer(r"\w+")
palavras_pro = token_pro.tokenize(" ".join(filtrado_pro))

fdist_pro = FreqDist(palavras_pro)
df_pro = fdist_pro.most_common(10)
df_pro = pd.DataFrame(df_pro, columns = ["Palavras", "Quantidade"]).sort_values(by=["Quantidade"],
                                                                                         ascending = False)
# Excluir a palavra aborto
df_pro = df_pro.drop(0)

In [ ]:
fdist_pro = FreqDist(palavras_pro)
df_pro = fdist_pro.most_common(20)
df_pro = pd.DataFrame(df_pro, columns = ["Palavras", "Quantidade"]).sort_values(by=["Quantidade"],
                                                                                         ascending = False)
# Excluir a palavra aborto
df_pro = df_pro.drop(0)
df_pro

In [ ]:
# Palavras mais comuns contra do aboro:

contra["tweet"]

texto_contra = ""
for i, r in contra.iterrows():
    texto_contra = texto_contra + r["tweet"] + "."
    texto_contra = texto_contra.lower()

token_contra = word_tokenize(texto_contra)

filtrado_contra = []
    
for p in token_contra:
    if p not in stop_words:
        filtrado_contra.append(p)

token_contra = RegexpTokenizer(r"\w+")
palavras_contra = token_contra.tokenize(" ".join(filtrado_contra))

fdist_contra = FreqDist(palavras_contra)
df_contra = fdist_contra.most_common(10)
df_contra = pd.DataFrame(df_contra, columns = ["Palavras", "Quantidade"]).sort_values(by=["Quantidade"],
                                                                                         ascending = False)
# Excluir a palavra aborto
df_contra = df_contra.drop(0)


In [ ]:
fdist_contra = FreqDist(palavras_contra)
df_contra = fdist_contra.most_common(20)
df_contra = pd.DataFrame(df_contra, columns = ["Palavras", "Quantidade"]).sort_values(by=["Quantidade"],
                                                                                         ascending = False)
# Excluir a palavra aborto
df_contra = df_contra.drop(0)
df_contra

### Visualizações

In [ ]:
# Nuvem de palavras

wordclound = WordCloud(stopwords = stop_words,
                      # background_color = "white",
                    #   colormap="Purples",
                       width = 3000,
                       height = 2000,
                       max_words = 300,
                      ).generate(texto)
plt.figure(figsize = (15,15))
plt.imshow(wordclound, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Frequência de palavras - geral

alt.data_transformers.disable_max_rows()

alt.Chart(df_palavras).mark_bar(size = 15, opacity=0.8).encode(
    x = alt.X("Palavras", title = "Palavras"),
    y = alt.Y("Quantidade", title = "Frequência"),
    tooltip = ["Palavras", "Quantidade"],
    color = alt.Color("Palavras", legend=None) 
).properties(width = 800, title = 'Principais palavras dos tweets e frequência que aparecem')

In [ ]:
# Frequência de palavras - a favor

alt.Chart(df_pro).mark_bar(size = 20, opacity=0.6).encode(
    x = alt.X("Quantidade", title = "Frequência"),
    y = alt.Y("Palavras", title = "Palavras"),
    tooltip = ["Palavras", "Quantidade"],
    color = alt.value("purple") 
).properties(height = 300, title = 'Principais palavras nos tweets classificados como favoráveis à legalização do aborto')

In [ ]:
# Frequência de palavras - contra

alt.Chart(df_contra).mark_bar(size = 20, opacity=0.7).encode(
    x = alt.X("Quantidade", title = "Frequência"),
    y = alt.Y("Palavras", title = "Palavras"),
    tooltip = ["Palavras", "Quantidade"],
    color = alt.value("red"), #legend=None) 
).properties(height = 300, title = 'Principais palavras nos tweets classificados como contrários à legalização do aborto')

In [ ]:
# Gráfico análise de sentimento

alt.Chart(df).mark_bar(size = 50, opacity=0.8).encode(
    x = alt.X("ME", title = ""),
    y = alt.Y("count()", title = "Quantidade de tweets"),
    tooltip = ["count()"],
    color = alt.Color("ME", legend = alt.Legend(title = "Posicionamento"))
).properties(width = 350, title = 'Análise de sentimento de tweets sobre "aborto"')
